# 01. PromptTemplate trong LangChain

## Mục tiêu
- Hiểu về PromptTemplate và tại sao cần sử dụng
- Tạo prompt động với các biến
- Sử dụng partial variables
- Format và validate prompts
- Best practices khi làm việc với prompts

## 1. Giới thiệu PromptTemplate

### Tại sao cần PromptTemplate?

Thay vì hardcode prompts:
```python
prompt = f"Translate '{text}' to {language}"
```

PromptTemplate giúp:
- **Tái sử dụng**: Dùng lại template cho nhiều inputs
- **Quản lý**: Tổ chức prompts một cách có hệ thống
- **Validation**: Kiểm tra input variables
- **Flexibility**: Dễ dàng thay đổi format

In [ ]:
# Cài đặt packages cần thiết
!pip install langchain langchain-anthropic langchain-openai

## 2. Basic PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

# Cách 1: Tạo từ string template
template = """You are a helpful assistant. 
Please answer the following question:

Question: {question}
Answer:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

# Format prompt với giá trị cụ thể
formatted_prompt = prompt.format(question="What is LangChain?")
print(formatted_prompt)

In [ ]:
# Cách 2: Sử dụng from_template (tự động detect variables)
prompt2 = PromptTemplate.from_template(
    "Translate the following {source_lang} text to {target_lang}: {text}"
)

print(f"Input variables detected: {prompt2.input_variables}")

# Format với multiple variables
result = prompt2.format(
    source_lang="English",
    target_lang="Vietnamese",
    text="Hello, how are you?"
)
print(f"\nFormatted prompt:\n{result}")

## 3. Advanced Template Features

In [ ]:
# Template với multiple lines và complex formatting
analysis_template = PromptTemplate.from_template("""
You are an expert {role} analyzing {topic}.

Context: {context}

Please provide:
1. A brief summary
2. Key insights
3. Recommendations

Focus on: {focus_areas}

Your analysis:
""")

# Format với detailed inputs
analysis = analysis_template.format(
    role="data scientist",
    topic="customer churn patterns",
    context="We have observed a 15% increase in churn rate last quarter",
    focus_areas="retention strategies and early warning signals"
)

print(analysis)

In [ ]:
# Template với conditional logic (using Jinja2 syntax)
from langchain.prompts import PromptTemplate

jinja_template = PromptTemplate(
    template="""
You are a {{ role }} assistant.
{% if detailed %}
Please provide a detailed analysis including:
- Background information
- Step-by-step reasoning
- Examples and references
{% else %}
Please provide a concise answer.
{% endif %}

Question: {{ question }}
""",
    input_variables=["role", "question", "detailed"],
    template_format="jinja2"
)

# Test với detailed=True
print("Detailed version:")
print(jinja_template.format(
    role="technical",
    question="Explain REST API",
    detailed=True
))

print("\n" + "="*50 + "\n")

# Test với detailed=False
print("Concise version:")
print(jinja_template.format(
    role="technical",
    question="Explain REST API",
    detailed=False
))

## 4. Partial Variables

Partial variables cho phép bạn "điền sẵn" một số biến trong template, tạo ra template mới với ít biến hơn.

In [ ]:
# Template với nhiều variables
base_template = PromptTemplate.from_template("""
You are a {language} programming expert.
Current date: {date}
User level: {level}

Question: {question}
""")

# Partial với một số biến cố định
python_expert = base_template.partial(
    language="Python",
    date="2024-03-15"
)

# Bây giờ chỉ cần provide 'level' và 'question'
print("Partial template input variables:", python_expert.input_variables)

result = python_expert.format(
    level="intermediate",
    question="How do decorators work?"
)
print("\nFormatted result:")
print(result)

In [ ]:
# Partial với functions (dynamic values)
from datetime import datetime

def get_current_time():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def get_system_info():
    return "LangChain Assistant v2.0"

# Template với dynamic partial values
dynamic_template = PromptTemplate(
    template="""
System: {system_info}
Timestamp: {current_time}
User: {user_name}

Query: {query}
""",
    input_variables=["user_name", "query"],
    partial_variables={
        "system_info": get_system_info,
        "current_time": get_current_time
    }
)

# Functions sẽ được gọi khi format
print(dynamic_template.format(
    user_name="Alice",
    query="What's the weather today?"
))

## 5. Template Composition

In [ ]:
# Kết hợp nhiều templates
system_template = PromptTemplate.from_template(
    "You are a {role} assistant specializing in {domain}."
)

instruction_template = PromptTemplate.from_template(
    "Please {action} the following {content_type}: {content}"
)

# Tạo template phức tạp từ các phần nhỏ
def create_full_prompt(role, domain, action, content_type, content):
    system_msg = system_template.format(role=role, domain=domain)
    instruction = instruction_template.format(
        action=action,
        content_type=content_type,
        content=content
    )
    
    return f"{system_msg}\n\n{instruction}"

# Test composition
full_prompt = create_full_prompt(
    role="technical",
    domain="software architecture",
    action="review and improve",
    content_type="system design",
    content="Microservices architecture for e-commerce platform"
)

print(full_prompt)

## 6. Validation và Error Handling

In [ ]:
# Template với validation
validated_template = PromptTemplate(
    template="Analyze {metric} for {period} in {region}",
    input_variables=["metric", "period", "region"],
    validate_template=True  # Default là True
)

# Test missing variable - sẽ raise error
try:
    result = validated_template.format(
        metric="sales",
        period="Q1 2024"
        # missing 'region'
    )
except KeyError as e:
    print(f"Validation error: {e}")

# Custom validation function
def validate_inputs(**kwargs):
    """Custom validation logic"""
    if "temperature" in kwargs:
        temp = kwargs["temperature"]
        if not isinstance(temp, (int, float)) or temp < 0 or temp > 2:
            raise ValueError(f"Temperature must be between 0 and 2, got {temp}")
    
    if "max_tokens" in kwargs:
        tokens = kwargs["max_tokens"]
        if not isinstance(tokens, int) or tokens < 1:
            raise ValueError(f"max_tokens must be positive integer, got {tokens}")
    
    return kwargs

# Template với custom validation
llm_template = PromptTemplate(
    template="Generate text with temperature={temperature} and max_tokens={max_tokens}: {prompt}",
    input_variables=["temperature", "max_tokens", "prompt"]
)

# Wrap formatting với validation
def safe_format(template, **kwargs):
    validated_kwargs = validate_inputs(**kwargs)
    return template.format(**validated_kwargs)

# Test valid inputs
try:
    result = safe_format(
        llm_template,
        temperature=0.7,
        max_tokens=100,
        prompt="Write a haiku"
    )
    print("Valid input - Success!")
    print(result)
except ValueError as e:
    print(f"Validation failed: {e}")

# Test invalid inputs
try:
    result = safe_format(
        llm_template,
        temperature=3.0,  # Invalid!
        max_tokens=100,
        prompt="Write a haiku"
    )
except ValueError as e:
    print(f"\nValidation failed as expected: {e}")

## 7. Real-world Examples

In [ ]:
# Example 1: Chain of Thought Prompt
cot_template = PromptTemplate.from_template("""
Question: {question}

Let's think step by step:
1. First, I need to understand what is being asked
2. Then, I'll identify the key information needed
3. Next, I'll work through the problem systematically
4. Finally, I'll provide the answer

Step-by-step solution:
""")

print(cot_template.format(
    question="If a train travels 120km in 2 hours, what is its average speed?"
))

In [ ]:
# Example 2: Few-shot Learning Template
few_shot_template = PromptTemplate.from_template("""
Task: {task_description}

Examples:
{examples}

Now, {task_action}: {input}
Output:
""")

# Prepare examples
sentiment_examples = """
Input: "This movie was fantastic! Best I've seen all year."
Output: Positive

Input: "Terrible service, would not recommend."
Output: Negative

Input: "The food was okay, nothing special."
Output: Neutral
"""

# Create sentiment analyzer
sentiment_prompt = few_shot_template.partial(
    task_description="Classify the sentiment of text as Positive, Negative, or Neutral",
    task_action="classify the sentiment of",
    examples=sentiment_examples
)

# Use for new input
print(sentiment_prompt.format(
    input="The product works as expected, good value for money."
))

In [ ]:
# Example 3: Multi-language Support Template
class MultilingualPromptTemplate:
    def __init__(self):
        self.templates = {
            "en": PromptTemplate.from_template(
                "You are a helpful assistant. Please {action}: {content}"
            ),
            "vi": PromptTemplate.from_template(
                "Bạn là trợ lý hữu ích. Vui lòng {action}: {content}"
            ),
            "fr": PromptTemplate.from_template(
                "Vous êtes un assistant utile. Veuillez {action}: {content}"
            )
        }
        
        self.action_translations = {
            "en": {"summarize": "summarize", "translate": "translate", "explain": "explain"},
            "vi": {"summarize": "tóm tắt", "translate": "dịch", "explain": "giải thích"},
            "fr": {"summarize": "résumer", "translate": "traduire", "explain": "expliquer"}
        }
    
    def format(self, language, action, content):
        if language not in self.templates:
            language = "en"  # fallback to English
        
        template = self.templates[language]
        localized_action = self.action_translations[language].get(action, action)
        
        return template.format(action=localized_action, content=content)

# Test multilingual templates
ml_prompt = MultilingualPromptTemplate()

for lang in ["en", "vi", "fr"]:
    print(f"\n{lang.upper()}:")
    print(ml_prompt.format(
        language=lang,
        action="summarize",
        content="The latest AI research paper"
    ))

## 8. Integration với LLMs

In [ ]:
# Tích hợp PromptTemplate với Claude
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage
import os

# Khởi tạo model (giả sử có API key)
# llm = ChatAnthropic(
#     model="claude-3-opus-20240229",
#     anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
# )

# Create a code review template
code_review_template = PromptTemplate.from_template("""
You are an expert {language} developer performing a code review.

Please review the following code for:
- Code quality and best practices
- Potential bugs or issues
- Performance considerations
- Security concerns

Code to review:
```{language}
{code}
```

Provide your review in the following format:
1. Summary
2. Issues found
3. Suggestions for improvement
4. Security considerations
""")

# Example usage
python_code = """
def get_user_data(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    result = database.execute(query)
    return result
"""

# Format prompt
review_prompt = code_review_template.format(
    language="Python",
    code=python_code
)

print("Generated prompt for LLM:")
print(review_prompt)

# Would send to LLM like this:
# response = llm.invoke([HumanMessage(content=review_prompt)])
# print(response.content)

## 9. Best Practices

### 1. Organize Templates

In [ ]:
# Tổ chức templates theo chức năng
class PromptLibrary:
    """Centralized prompt management"""
    
    # Analysis prompts
    ANALYZE_DATA = PromptTemplate.from_template(
        "Analyze the following {data_type} data and provide insights: {data}"
    )
    
    # Generation prompts
    GENERATE_CODE = PromptTemplate.from_template(
        "Generate {language} code that {functionality}. Requirements: {requirements}"
    )
    
    # Translation prompts
    TRANSLATE = PromptTemplate.from_template(
        "Translate from {source_lang} to {target_lang}: {text}"
    )
    
    @classmethod
    def get_prompt(cls, prompt_name: str) -> PromptTemplate:
        """Get prompt by name"""
        return getattr(cls, prompt_name, None)

# Usage
prompt = PromptLibrary.GENERATE_CODE
result = prompt.format(
    language="Python",
    functionality="sorts a list of dictionaries by a specific key",
    requirements="Handle missing keys gracefully, support reverse sorting"
)
print(result)

### 2. Version Control cho Prompts

In [ ]:
from datetime import datetime
from typing import Dict, Optional

class VersionedPromptTemplate:
    """Prompt template with version control"""
    
    def __init__(self, name: str):
        self.name = name
        self.versions: Dict[str, PromptTemplate] = {}
        self.current_version: Optional[str] = None
    
    def add_version(self, version: str, template: PromptTemplate, set_current: bool = True):
        """Add a new version of the prompt"""
        self.versions[version] = template
        if set_current:
            self.current_version = version
    
    def get_version(self, version: Optional[str] = None) -> PromptTemplate:
        """Get specific version or current version"""
        if version is None:
            version = self.current_version
        return self.versions.get(version)
    
    def format(self, version: Optional[str] = None, **kwargs) -> str:
        """Format the prompt with given version"""
        template = self.get_version(version)
        if template is None:
            raise ValueError(f"Version {version} not found")
        return template.format(**kwargs)

# Example usage
summarizer = VersionedPromptTemplate("summarizer")

# Version 1: Simple
summarizer.add_version(
    "v1.0",
    PromptTemplate.from_template("Summarize: {text}")
)

# Version 2: With length control
summarizer.add_version(
    "v2.0",
    PromptTemplate.from_template(
        "Summarize the following text in {max_words} words or less: {text}"
    )
)

# Version 3: With style
summarizer.add_version(
    "v3.0",
    PromptTemplate.from_template("""
    Summarize the following text in {style} style.
    Maximum length: {max_words} words
    Focus on: {focus}
    
    Text: {text}
    """)
)

# Test different versions
text = "LangChain is a framework for developing applications powered by language models..."

print("Version 1.0:")
print(summarizer.format(version="v1.0", text=text))
print("\n" + "="*50 + "\n")

print("Current version (v3.0):")
print(summarizer.format(
    text=text,
    style="technical",
    max_words=50,
    focus="key features and benefits"
))

## 10. Performance Tips

In [ ]:
# Caching formatted prompts
from functools import lru_cache
import hashlib

class CachedPromptTemplate:
    """PromptTemplate with caching for expensive operations"""
    
    def __init__(self, template: PromptTemplate):
        self.template = template
        self._cache = {}
    
    def _get_cache_key(self, **kwargs) -> str:
        """Generate cache key from kwargs"""
        # Sort keys for consistent hashing
        sorted_items = sorted(kwargs.items())
        key_str = str(sorted_items)
        return hashlib.md5(key_str.encode()).hexdigest()
    
    def format(self, use_cache: bool = True, **kwargs) -> str:
        """Format with optional caching"""
        if not use_cache:
            return self.template.format(**kwargs)
        
        cache_key = self._get_cache_key(**kwargs)
        
        if cache_key in self._cache:
            print(f"Cache hit for key: {cache_key[:8]}...")
            return self._cache[cache_key]
        
        print(f"Cache miss for key: {cache_key[:8]}...")
        result = self.template.format(**kwargs)
        self._cache[cache_key] = result
        return result
    
    def clear_cache(self):
        """Clear the cache"""
        self._cache.clear()

# Test caching
expensive_template = PromptTemplate.from_template(
    "Process {data} with method {method} and parameters {params}"
)

cached_template = CachedPromptTemplate(expensive_template)

# First call - cache miss
result1 = cached_template.format(
    data="customer_data",
    method="analysis",
    params="detailed"
)

# Second call with same params - cache hit
result2 = cached_template.format(
    data="customer_data",
    method="analysis",
    params="detailed"
)

# Different params - cache miss
result3 = cached_template.format(
    data="product_data",
    method="analysis",
    params="summary"
)

## Tổng kết

### Key Takeaways:
1. **PromptTemplate** giúp quản lý prompts một cách có hệ thống
2. **Variables** cho phép tái sử dụng templates với different inputs
3. **Partial** variables giúp tạo specialized templates
4. **Validation** đảm bảo input đúng format
5. **Composition** cho phép xây dựng complex prompts từ simple parts

### Best Practices:
- Organize prompts trong classes hoặc modules
- Version control cho prompts quan trọng
- Cache khi cần performance
- Validate inputs trước khi format
- Use meaningful variable names

### Next Steps:
- Explore ChatPromptTemplate cho conversations
- Learn about FewShotPromptTemplate
- Integrate với LangChain chains và agents